# Perform read smoothing then assemble with LJA

In [1]:
%run "Header.ipynb"

In [5]:
import os
import time
import pickle
import pysam
import skbio
from collections import defaultdict
from linked_mutations_utils import find_mutated_positions

## 1. Smooth reads

Lots of this code is duplicated from the `Phasing-01-MakeGraph.ipynb` notebook in this folder.

In [11]:
bf = pysam.AlignmentFile("../main-workflow/output/fully-filtered-and-sorted-aln.bam", "rb")
output_dir = "phasing-data/smoothed-reads/"

# verbose?
no_indoor_voice = False

def write_out_reads(filepath, readname2seq):
    # Notably, this uses the "a" (append) method in order to add to the end of a file
    with open(filepath, "a") as of:
        for readname in readname2seq:
            # Write out both the header and the sequence for each read
            of.write(f">{readname}\n{str(readname2seq[readname])}\n")
            
ALN_UPDATE_FREQ = 1000
ALN_BUFFER_FREQ = 1000
            
t1 = time.time()
for seq in SEQS:
    fasta = skbio.DNA.read(f"../seqs/{seq}.fasta")
    
    output_smoothed_reads_file = os.path.join(output_dir, f"{seq}_smoothed_reads.fasta")
    
    # Identify all (0-indexed, so compatible with skbio / pysam!)
    # mutated positions in this genome up front to save time.
    #
    # Equivalently, we could also just take in an arbitrary VCF as input
    # (e.g. one produced from another variant calling tool), although we'd
    # need to be careful to only include SNVs and not indels/etc...
    
    print(f"Identifying mutated positions in genome {seq2name[seq]}...")
    mutated_positions = find_mutated_positions(seq)
    print(f"Found {len(mutated_positions):,} mutated positions in {seq2name[seq]}.")
    
    print("Going through these positions...")
    
    # This should already be implicitly sorted, I think, but the code below relies on mutated_positions being
    # in the exact same order as expected. So we may as well be paranoid.
    mutated_positions = sorted(mutated_positions)
    
    # Instead of just writing out every smoothed alignment as soon as we generate it, we build up a "buffer"
    # of these alignments and then write a bunch out at once. This way we limit slowdown due to constantly
    # having to open/close files. I don't really have a good source for this as best practice, but I remembered
    # to do it while writing this code, so somewhere in College Park the CS faculty at Maryland are smiling
    #
    # Also fyi this maps read name to smoothed alignment (well, at this point, just read) sequence. The read name
    # is useful to preserve in fasta files so we have some idea of provenance (where smoothed reads came from)
    smoothed_aln_buffer = {}
    
    # The first time we see an alignment of a read, it's 1; if we see a supp aln of this read, it's 2; etc.
    # Lets us distinguish alignments with different names
    readname2freq_so_far = defaultdict(int)
    
    # Go through all linear alignments of each read to this genome, focusing (for now) on just the primary
    # alignments...
    ts1 = time.time()
    for ai, aln in enumerate(bf.fetch(seq), 1):
        
        if ai % ALN_UPDATE_FREQ == 0:
            print(
                f"\tOn aln {ai:,} in seq {seq2name[seq]}. "
                f"Time spent on {seq2name[seq]} so far: {time.time() - ts1:,.2f} sec."
            )
            
        if aln.is_secondary:
            raise ValueError(
                "Not to get political or anything, but you should've already filtered secondary alns out"
            )
            
        # Note that supplementary alignments are ok, though! We implicitly handle these here.
        #
        # Different alignments of the same read will have different new_readnames, because we're gonna
        # be treating them as distinct "reads". We should have already filtered reference-overlapping
        # supp alns so this shouldn't be a problem
        
        readname = aln.query_name
        readname2freq_so_far[readname] += 1
        new_readname = f"{readname}_{readname2freq_so_far[readname]}"
        
        # should never happen
        if new_readname in smoothed_aln_buffer:
            raise ValueError("This exact read alignment has already been smoothed? Weird.")
            
        # Figure out where on the MAG this alignment "hits." These are 0-indexed positions from Pysam.
        # (reference_end points to the position after the actual final position, since these are designed to
        # be interoperable with Python's half-open intervals.)
        #
        # Of course, there likely will be indels within this range: we're purposefully ignoring those here.
        ref_start = aln.reference_start
        ref_end = aln.reference_end - 1
        
        # This should never happen (TM)
        if ref_start >= ref_end:
            # Du sollst jetzt mit Gott sprechen.
            raise ValueError(
                f"Ref start {ref_start:,} >= ref end {ref_end:,} for read {new_readname}?"
            )
        
        # Smoothed sequence; we'll edit this so that if this read has (mis)matches to any called mutated
        # positions, these positions are updated with the read's aligned nucleotides at these positions.
        smoothed_aln_seq = fasta[ref_start: ref_end + 1]
        
        # just for debugging: track the exact edits made to smoothed_aln_seq
        replacements_made = {}
        
        ap = aln.get_aligned_pairs(matches_only=True)
        
        # Iterating through the aligned pairs is expensive. Since read lengths are generally in the thousands
        # to tens of thousands of bp (which is much less than the > 1 million bp length of any bacterial genome),
        # we set things up so that we only iterate through the aligned pairs once. We maintain an integer, mpi,
        # that is a poor man's "pointer" to an index in mutated_positions.
        
        mpi = 0
        
        # Go through this aln's aligned pairs. As we see each pair, compare the pair's reference position
        # (refpos) to the mpi-th mutated position (herein referred to as "mutpos").
        #
        # If refpos >  mutpos, increment mpi until refpos <= mutpos (stopping as early as possible).
        # If refpos == mutpos, we have a match! Update readname2mutpos2ismutated[mutpos] based on
        #                      comparing the read to the reference at the aligned positions.
        # If refpos <  mutpos, continue to the next pair.
        
        for pair in ap:
            
            refpos = pair[1]
            mutpos = mutated_positions[mpi]
            
            no_mutations_to_right_of_here = False
            
            # Increment mpi until we get to the next mutated position at or after the reference pos for this
            # aligned pair (or until we run out of mutated positions).
            while refpos > mutpos:
                mpi += 1
                if mpi < len(mutated_positions):
                    mutpos = mutated_positions[mpi]
                else:
                    no_mutations_to_right_of_here = True
                    break
            
            # I expect this should happen only for reads aligned near the right end of the genome.
            if no_mutations_to_right_of_here:
                break
            
            # If the next mutation occurs after this aligned pair, continue on to a later pair.
            if refpos < mutpos:
                continue
                
            # If we've made it here, refpos == mutpos!
            # (...unless I messed something up in how I designed this code.)
            if refpos != mutpos:
                raise ValueError("This should never happen!")
                
            # Finally, get the nucleotide aligned to this mutated position from this read.
            readpos = pair[0]
            read_nt = aln.query_sequence[readpos]
            
            ref_nt = str(fasta[mutpos])
            # We don't need to do anything if this read already matches the reference MAG at this position
            if read_nt == ref_nt:
                if no_indoor_voice:
                    print(f"Read {new_readname} matches ref at mutpos {mutpos + 1:,}: both {read_nt}")
            else:
                # Record this specific "allele" for this read.
                relative_pos_on_aln = mutpos - ref_start
                smoothed_aln_seq = smoothed_aln_seq.replace([relative_pos_on_aln], read_nt)
                replacements_made[relative_pos_on_aln] = read_nt
                if no_indoor_voice:
                    print(
                        f"Read {new_readname} mismatches ref at mutpos {mutpos + 1:,}: "
                        f"ref = {ref_nt}, read = {read_nt}"
                    )
                
        if no_indoor_voice:
            print(f"Read {new_readname} required {len(replacements_made):,} replacements!")
        
        # Now that we've finished processing all called mutations that this alignment spans, prepare it
        # to be written out to a FASTA file. See comments above on smoothed_aln_buffer, and why we don't
        # just write everything out as soon as it's ready.
        #
        # (Also, we've already guaranteed readname isn't already in smoothed_aln_buffer, so no need to worry
        # about accidentally overwriting something from earlier.)
        smoothed_aln_buffer[new_readname] = smoothed_aln_seq
        
        # Notably, we don't necessarily write out *exactly* ALN_BUFFER_FREQ reads at once -- skipping alignments
        # due to them being supplementary, etc. (actually no need for an "etc.", that's literally the only
        # possible reason as of writing, but whatever) doesn't stop ai from going up. Shouldn't make a difference
        # unless we have a zillion supplementary alignments.
        if ai % ALN_BUFFER_FREQ == 0:
            write_out_reads(output_smoothed_reads_file, smoothed_aln_buffer)
            # Clear the buffer
            smoothed_aln_buffer = {}
        
    # We're probably going to have left over smoothed reads that we still haven't written out, unless things
    # worked out so that on the final alignment we saw ai was exactly divisible by ALN_BUFFER_FREQ (and that's
    # pretty unlikely unless you set the buffer freq to a low number). So make one last dump of the buffer.
    if len(smoothed_aln_buffer) > 0:
        write_out_reads(output_smoothed_reads_file, smoothed_aln_buffer)
    
    print(f"Done with {seq}! Took {time.time() - ts1:,.2f} sec.")
        
print(f"Time taken: {time.time() - t1:,} sec.")

Identifying mutated positions in genome CAMP...
Found 284 mutated positions in CAMP.
Going through these positions...
Read m54033_180919_161442/34472121/ccs_1 matches ref at mutpos 1,596: both A
Read m54033_180919_161442/34472121/ccs_1 required 0 replacements!
Read m54033_180919_161442/44106480/ccs_1 matches ref at mutpos 1,596: both A
Read m54033_180919_161442/44106480/ccs_1 required 0 replacements!
Read m54033_180919_161442/68944731/ccs_1 matches ref at mutpos 1,596: both A
Read m54033_180919_161442/68944731/ccs_1 required 0 replacements!
Read m54033_180919_161442/70123867/ccs_1 matches ref at mutpos 1,596: both A
Read m54033_180919_161442/70123867/ccs_1 required 0 replacements!
Read m54033_180919_161442/16974032/ccs_1 matches ref at mutpos 1,596: both A
Read m54033_180919_161442/16974032/ccs_1 required 0 replacements!
Read m54033_180919_161442/20841189/ccs_1 matches ref at mutpos 1,596: both A
Read m54033_180919_161442/20841189/ccs_1 required 0 replacements!
Read m54033_180919_16144

Read m54337_181126_072918/53543053/ccs_1 matches ref at mutpos 1,596: both A
Read m54337_181126_072918/53543053/ccs_1 required 0 replacements!
Read m54337_181126_072918/60162840/ccs_1 required 0 replacements!
Read m54337_181126_072918/60359374/ccs_1 matches ref at mutpos 1,596: both A
Read m54337_181126_072918/60359374/ccs_1 required 0 replacements!
Read m54337_181126_072918/64750252/ccs_1 matches ref at mutpos 1,596: both A
Read m54337_181126_072918/64750252/ccs_1 required 0 replacements!
Read m54337_181126_072918/69206236/ccs_1 matches ref at mutpos 1,596: both A
Read m54337_181126_072918/69206236/ccs_1 required 0 replacements!
Read m54337_181125_110904/24838283/ccs_1 matches ref at mutpos 1,596: both A
Read m54337_181125_110904/24838283/ccs_1 required 0 replacements!
Read m54337_181125_110904/26935554/ccs_1 matches ref at mutpos 1,596: both A
Read m54337_181125_110904/26935554/ccs_1 required 0 replacements!
Read m54337_181125_110904/35390356/ccs_1 matches ref at mutpos 1,596: both A

Read m54033_181204_171640/44302408/ccs_1 matches ref at mutpos 1,596: both A
Read m54033_181204_171640/44302408/ccs_1 required 0 replacements!
Read m54033_181204_171640/44368619/ccs_1 matches ref at mutpos 1,596: both A
Read m54033_181204_171640/44368619/ccs_1 required 0 replacements!
Read m54033_181204_171640/58982825/ccs_1 matches ref at mutpos 1,596: both A
Read m54033_181204_171640/58982825/ccs_1 required 0 replacements!
Read m54033_181204_171640/8586075/ccs_1 matches ref at mutpos 1,596: both A
Read m54033_181204_171640/8586075/ccs_1 required 0 replacements!
Read m54033_181204_171640/10027692/ccs_1 matches ref at mutpos 1,596: both A
Read m54033_181204_171640/10027692/ccs_1 required 0 replacements!
Read m54033_181204_171640/60687006/ccs_1 matches ref at mutpos 1,596: both A
Read m54033_181204_171640/60687006/ccs_1 required 0 replacements!
Read m54033_181204_171640/65798600/ccs_1 matches ref at mutpos 1,596: both A
Read m54033_181204_171640/65798600/ccs_1 required 0 replacements!
R

Read m54033_181217_171601/71369168/ccs_1 matches ref at mutpos 1,596: both A
Read m54033_181217_171601/71369168/ccs_1 required 0 replacements!
Read m54033_181217_171601/13763188/ccs_1 matches ref at mutpos 1,596: both A
Read m54033_181217_171601/13763188/ccs_1 required 0 replacements!
Read m54033_181217_171601/22086085/ccs_1 matches ref at mutpos 1,596: both A
Read m54033_181217_171601/22086085/ccs_1 required 0 replacements!
Read m54033_181217_171601/23331592/ccs_1 matches ref at mutpos 1,596: both A
Read m54033_181217_171601/23331592/ccs_1 required 0 replacements!
Read m54033_181217_171601/30147360/ccs_1 matches ref at mutpos 1,596: both A
Read m54033_181217_171601/30147360/ccs_1 required 0 replacements!
Read m54033_181217_171601/31916380/ccs_1 matches ref at mutpos 1,596: both A
Read m54033_181217_171601/31916380/ccs_1 required 0 replacements!
Read m54033_181217_171601/33555361/ccs_1 matches ref at mutpos 1,596: both A
Read m54033_181217_171601/33555361/ccs_1 required 0 replacements!

Read m54337_181219_124328/36307867/ccs_1 matches ref at mutpos 1,596: both A
Read m54337_181219_124328/36307867/ccs_1 required 0 replacements!
Read m54337_181219_124328/42795542/ccs_1 matches ref at mutpos 1,596: both A
Read m54337_181219_124328/42795542/ccs_1 required 0 replacements!
Read m54337_181219_124328/43910117/ccs_1 matches ref at mutpos 1,596: both A
Read m54337_181219_124328/43910117/ccs_1 required 0 replacements!
Read m54337_181219_124328/54132912/ccs_1 matches ref at mutpos 1,596: both A
Read m54337_181219_124328/54132912/ccs_1 required 0 replacements!
Read m54337_181219_124328/58786647/ccs_1 matches ref at mutpos 1,596: both A
Read m54337_181219_124328/58786647/ccs_1 required 0 replacements!
Read m54337_181219_124328/67895955/ccs_1 matches ref at mutpos 1,596: both A
Read m54337_181219_124328/67895955/ccs_1 required 0 replacements!
Read m54337_181219_124328/74318582/ccs_1 required 0 replacements!
Read m54337_181219_124328/74645971/ccs_1 matches ref at mutpos 1,596: both A

...

Read m54337U_200211_145859/11535925/ccs_1 matches ref at mutpos 20,103: both A
Read m54337U_200211_145859/11535925/ccs_1 matches ref at mutpos 27,554: both G
Read m54337U_200211_145859/11535925/ccs_1 required 0 replacements!
Read m54337U_200220_195233/32310216/ccs_1 matches ref at mutpos 20,103: both A
Read m54337U_200220_195233/32310216/ccs_1 matches ref at mutpos 27,554: both G
Read m54337U_200220_195233/32310216/ccs_1 required 0 replacements!
Read m54337U_200220_195233/77400684/ccs_1 matches ref at mutpos 20,103: both A
Read m54337U_200220_195233/77400684/ccs_1 matches ref at mutpos 27,554: both G
Read m54337U_200220_195233/77400684/ccs_1 required 0 replacements!
Read m54337U_200213_024013/93456777/ccs_1 matches ref at mutpos 20,103: both A
Read m54337U_200213_024013/93456777/ccs_1 matches ref at mutpos 27,554: both G
Read m54337U_200213_024013/93456777/ccs_1 required 0 replacements!
Read m54337U_200214_212611/107675965/ccs_1 matches ref at mutpos 20,103: both A
Read m54337U_200214_

Read m54337U_200222_075656/96273306/ccs_1 matches ref at mutpos 20,103: both A
Read m54337U_200222_075656/96273306/ccs_1 matches ref at mutpos 27,554: both G
Read m54337U_200222_075656/96273306/ccs_1 required 0 replacements!
Read m54337U_200223_200916/144244866/ccs_1 matches ref at mutpos 20,103: both A
Read m54337U_200223_200916/144244866/ccs_1 required 0 replacements!
Read m54337U_200227_201603/114034033/ccs_1 matches ref at mutpos 20,103: both A
Read m54337U_200227_201603/114034033/ccs_1 matches ref at mutpos 27,554: both G
Read m54337U_200227_201603/114034033/ccs_1 required 0 replacements!
Read m54337U_200214_212611/61146520/ccs_1 matches ref at mutpos 20,103: both A
Read m54337U_200214_212611/61146520/ccs_1 matches ref at mutpos 27,554: both G
Read m54337U_200214_212611/61146520/ccs_1 required 0 replacements!
Read m54337U_200222_075656/144114593/ccs_1 matches ref at mutpos 20,103: both A
Read m54337U_200222_075656/144114593/ccs_1 required 0 replacements!
Read m54337U_200220_195233

Read m54337U_200220_195233/160303896/ccs_1 matches ref at mutpos 20,103: both A
Read m54337U_200220_195233/160303896/ccs_1 required 0 replacements!
Read m54337U_200211_145859/14288256/ccs_1 matches ref at mutpos 20,103: both A
Read m54337U_200211_145859/14288256/ccs_1 matches ref at mutpos 27,554: both G
Read m54337U_200211_145859/14288256/ccs_1 required 0 replacements!
Read m54337U_200213_024013/144967473/ccs_1 matches ref at mutpos 20,103: both A
Read m54337U_200213_024013/144967473/ccs_1 matches ref at mutpos 27,554: both G
Read m54337U_200213_024013/144967473/ccs_1 required 0 replacements!
Read m54337U_200203_184822/125764529/ccs_1 matches ref at mutpos 20,103: both A
Read m54337U_200203_184822/125764529/ccs_1 required 0 replacements!
Read m54337U_200213_024013/54658460/ccs_1 matches ref at mutpos 20,103: both A
Read m54337U_200213_024013/54658460/ccs_1 matches ref at mutpos 27,554: both G
Read m54337U_200213_024013/54658460/ccs_1 required 0 replacements!
Read m54337U_200220_195233

Read m54337U_200227_201603/126419002/ccs_1 matches ref at mutpos 20,103: both A
Read m54337U_200227_201603/126419002/ccs_1 matches ref at mutpos 27,554: both G
Read m54337U_200227_201603/126419002/ccs_1 required 0 replacements!
Read m54337U_200211_145859/145425896/ccs_1 matches ref at mutpos 20,103: both A
Read m54337U_200211_145859/145425896/ccs_1 matches ref at mutpos 27,554: both G
Read m54337U_200211_145859/145425896/ccs_1 required 0 replacements!
Read m54033_181204_171640/72090096/ccs_1 matches ref at mutpos 20,103: both A
Read m54033_181204_171640/72090096/ccs_1 matches ref at mutpos 27,554: both G
Read m54033_181204_171640/72090096/ccs_1 required 0 replacements!
Read m54337U_200213_024013/148111572/ccs_1 matches ref at mutpos 20,103: both A
Read m54337U_200213_024013/148111572/ccs_1 matches ref at mutpos 27,554: both G
Read m54337U_200213_024013/148111572/ccs_1 required 0 replacements!
Read m54337U_200223_200916/36897966/ccs_1 matches ref at mutpos 20,103: both A
Read m54337U_20

KeyboardInterrupt: 

## 2. Run LJA on these smoothed reads

In [ ]:
!/home/mfedarko/software/LJA/bin/lja \
    --reads phasing-data/smoothed-reads/edge_6104_smoothed_reads.fasta \
    --output-dir phasing-data/smoothed-reads/edge_6104_lja

!/home/mfedarko/software/LJA/bin/lja \
    --reads phasing-data/smoothed-reads/edge_1671_smoothed_reads.fasta \
    --output-dir phasing-data/smoothed-reads/edge_1671_lja

!/home/mfedarko/software/LJA/bin/lja \
    --reads phasing-data/smoothed-reads/edge_2358_smoothed_reads.fasta \
    --output-dir phasing-data/smoothed-reads/edge_2358_lja